In [1]:
from data import Data
from config import read_config
from feature_engineer import FeatureEngineer
from model import Model
from stack import Stacker

In [2]:
data = Data()
data.data_peek()


Train:
--------------------------------------------------

   PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5       3                           Allen, Mr. William Henry   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S  
1  female  38.0      1      0          PC 17599  71.2833   C85        C  
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S  
3  female  35.0      1      0            113803  53.1000  C123        S  
4    male  35.0      0      0            373450   8.0500   NaN        S  

Test:
--------------------------

In [17]:
import re
name= data.train['Name']
name = name.apply(lambda x: re.split('\W+', x)[1])
name.head()

0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: Name, dtype: object

In [20]:
name.value_counts()

Mr             493
Miss           172
Mrs            119
Master          39
Rev              6
Dr               6
y                4
Planke           3
Impe             3
Brien            3
Major            2
Mlle             2
Franchi          2
Yarred           2
Gordon           2
Carlo            1
Jensen           1
Col              1
Maeglin          1
Lambert          1
Cruyssen         1
Walle            1
Connor           1
Connell          1
Bratthammer      1
Steffansson      1
Pelsmaeker       1
Messemaeker      1
Driscoll         1
Billiard         1
Shawah           1
Mme              1
the              1
Leary            1
Sullivan         1
Jonkheer         1
Mulder           1
Stehli           1
Velde            1
Blumer           1
der              1
Ms               1
Melkebeke        1
Heilmann         1
Don              1
Steen            1
Dwyer            1
Capt             1
Name: Name, dtype: int64

In [35]:
import numpy as np
name[name == 'Mr'] = 1
name[(name == 'Miss') | (name == 'Mlle')] = 2
name[(name == 'Mrs') | (name == 'Mme')] = 3
name[name == 'Master'] = 4

In [38]:
name[(name != 1) & (name != 2) & (name != 3) & (name != 4)] = 0

In [44]:
import pandas as pd
name = pd.Series(name, dtype = int)

In [9]:
 param = read_config()

{'features': {'Age': 'fillna_scale',
              'Cabin': 'first_letter',
              'Embarked': 'get_dummies',
              'Fare': 'fillna_scale',
              'Parch': 'good',
              'Pclass': 'good',
              'Sex': 'get_dummies',
              'SibSp': 'good'},
 'model': {'knn128': {'n_neighbors': 128},
           'knn32': {'n_neighbors': 32},
           'knn64': {'n_neighbors': 64},
           'logit': {'C': 0.525},
           'logit_grid': {'C': [0.45, 0.475, 0.525, 0.55]},
           'rf': {'max_features': 'sqrt',
                  'min_samples_leaf': 45,
                  'min_samples_split': 3,
                  'n_estimators': 150},
           'rf_grid': {'max_features': ['sqrt'],
                       'min_samples_leaf': [45, 50, 55],
                       'min_samples_split': [2, 3],
                       'n_estimators': [150]},
           'svm': {'C': 0.75, 'probability': True},
           'svm_grid': {'C': [0.6, 0.7, 0.8, 0.9], 'kernel': ['rbf']},
 

In [4]:
features = FeatureEngineer(data, param['features'])

In [5]:
model_xgb = Model(features, 'xgb', param['model'])
    #model_xgb.cross_validation_all(5, 'accuracy', False, True)
model_rf = Model(features, 'rf', param['model'])
    # model_rf.cross_validation_all(5, 'accuracy', False, True)
    # model_rf.grid_search('accuracy', 5)
model_knn32 = Model(features, 'knn32', param['model'])
    # pred = model_xgb.grid_search('accuracy', 5)
model_knn128 = Model(features, 'knn128', param['model'])
    # model_knn128.cross_validation_all(5, 'accuracy', False, True)
model_logit = Model(features, 'logit', param['model'])
    # model_logit.grid_search('accuracy', 5)
model_svm = Model(features, 'svm', param['model'])

In [6]:
model_list = [model_xgb,
              model_rf,
              model_knn32,
              model_knn128,
              model_logit,
              model_svm]
    #
stack_model_xgb = Model(features, 'xgb', param['stack_model'])
stack_model_list = [stack_model_xgb]

In [7]:
stack = Stacker(features, 3, model_list, stack_model_list, 2)


Start feature stacking for xgb
logloss: 0.472242
Start feature stacking for rf
logloss: 0.497841
Start feature stacking for knn32
logloss: 0.477353
Start feature stacking for knn128
logloss: 0.531765
Start feature stacking for logit
logloss: 0.456543
Start feature stacking for svm
logloss: 0.475507

Meta_feature created with shape (891, 12) in train and (418, 12) in test

model xgb:
accuracy: 0.808081


In [8]:
pred = stack.stack_model_grid_search(0)


Start grid search for xgb
{'max_depth': [3, 4], 'n_estimators': [10, 50, '100 150']}


TypeError: range() integer end argument expected, got str.